<a href="https://colab.research.google.com/github/ayushgupta799/Amazon-Product-Sentiment/blob/main/AnalysingProductSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U turicreate

In [ ]:
import turicreate

In [ ]:
products = turicreate.SFrame('/content/drive/My Drive/Colab Notebooks/amazon_baby.sframe/')

In [ ]:
products

In [ ]:
products.head()
products['word_count'] = turicreate.text_analytics.count_words(products['review'])
products.head()

In [ ]:
products['name'].show()

In [ ]:
giraffe_reviews = products[products['name']== 'Vulli Sophie the Giraffe Teether']
len(giraffe_reviews)

In [ ]:
turicreate.visualization.histogram(giraffe_reviews['rating'])

**build a sentiment classifier**

In [ ]:
products['rating'].show()

In [ ]:
products = products[products['rating'] != 3]
products['sentiment']= products['rating'] >= 4
products.head()

In [ ]:
train_data,test_data = products.random_split(.8,seed=0)

In [ ]:
sentiment_model= turicreate.logistic_classifier.create(train_data,
                                                       target = 'sentiment',
                                                       features = ['word_count'],
                                                       validation_set= test_data)

In [ ]:
sentiment_model.training_accuracy

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
roc_eval = sentiment_model.evaluate(test_data, metric='roc_curve')


In [ ]:
plt.scatter(roc_eval['roc_curve']['fpr'],
            roc_eval['roc_curve']['tpr'],  
            label='ROC Curve')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews,output_type = 'probability')

In [ ]:
giraffe_reviews.head()

In [ ]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment',ascending=False)
giraffe_reviews.head()

In [ ]:
giraffe_reviews[2]['review']

In [ ]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [ ]:
for each in selected_words:
  products[each]= products['word_count'].apply(lambda counts:\
                                               counts.get(each,0))

In [ ]:
for each in selected_words:
  print (each,products[each].sum())

In [ ]:
train_data,test_data = products.random_split(.8,seed=0)

In [ ]:
selected_words_model= turicreate.logistic_classifier.create(train_data,
                                                       target = 'sentiment',
                                                       features = selected_words,
                                                       validation_set= test_data)

In [ ]:
selected_words_model.training_accuracy

In [ ]:
selected_words_model.coefficients

In [ ]:
btdc_reviews = products[products['name']== 'Baby Trend Diaper Champ']

In [ ]:
btdc_reviews['predicted_sentiment'] = sentiment_model.predict(btdc_reviews,output_type = 'probability')
btdc_reviews= btdc_reviews.sort('predicted_sentiment',ascending=False)
btdc_reviews

In [ ]:
btdc_reviews['predicted_sentiment'] = selected_words_model.predict(btdc_reviews,output_type = 'probability')
btdc_reviews= btdc_reviews.sort('predicted_sentiment',ascending=False)
btdc_reviews

In [ ]:
selected_words_model.predict(btdc_reviews[0:1], output_type='probability')